# Lab 1 : Visualization and Data Processing

## Group 4 :  Simerpreet Reddy, Rudranil Mitra, Samuel VonPaays Soh, Kristi Herman

---
## <font color='darkgreen'> Business Understanding </font>

The source selected for this analysis is a Craigslist used car listings dataset from Kaggle:  https://www.kaggle.com/austinreese/craigslist-carstrucks-data<br>  The author of this dataset used web scraping algorithms to scrape the information for used vehicles from all regional Craigslist's sites in the U.S that were posted between 10/28/2020 and 12/3/2020.  The original dataset contained <b>458,213 rows</b> with <b>26 fields</b>. 


#### <font color='green'>Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?)</font>
The purpose of this dataset is to collect data for all used cars and trucks for sale in the U.S. that are listed on Craigslist regional sites.  This dataset was selected to model and predict used car prices and to classify price ranges by categories.  Included in the dataset were other types of vehicles such as buses and motorcycles.  Those records were removed to limit the analysis to cars and trucks only. 


#### <font color='green'>Describe how you would define and measure the outcomes from the dataset. That is, why is this data important and how do you know if you have mined useful knowledge from the dataset? </font>

We will know that we have mined useful knowledge if we are able to come up with a good prediction for used car prices and a classifier for price range predictions. In other words, a good model will be able to find relationships among the predictors and the response variables.  

#### <font color='green'>How would you measure the effectiveness of a good prediction algorithm? Be specific. </font>

Our goal is to run both both regression and classification algorithms on this dataset. For the regression problem of predicting car prices, RMSE along with k-fold cross validation will be used to determine the effectiveness of the prediction. For the classification problem, k-fold cross validation with accuracy, precision, recall, True Positive Rate, and False Positive Rate will be used to measure the effectiveness of the prediction model. The value for 'k' in cross validation will be determined through hyperparameter tuning during the model building process.

## Data Meaning Type

*Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file.*

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import requests
import json
import time
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
import folium

In [ ]:
#import vehicle dataset
orig_df = pd.read_csv('data/vehicles.csv') 

<blockquote>The original dataset contained <b>458,213 rows</b> with <b>26 fields</b>. 

In [ ]:
# Copy original file to car_df to avoid reloading the large file, keep the original dataset in case it is needed. 
car_df = orig_df.copy()

#info
car_df.info()

<blockquote>These fields were immediately dropped for the following reasons:<br>
1) The features were not useful for the two prediction problems in this analysis (id, Unnamed:0, url, image_url, region_url, description)<br>
2) There were too many missing values that could not be easily derived (size) <br> 
3) The features provided similar location information to the "state" feature (region, lat, long were dropped and state was kept)

In [ ]:
car_df = car_df.drop(columns=['id','Unnamed: 0','url','image_url','region_url','description','size','region','lat','long'])

<blockquote>This is a more detailed description of the fields that will be used for explanatory and response variables after some transformations are applied in <a href='#dataquality'>Data Quality</a> .  The first two variables listed are the ones that will be predicted - price and price_range. </blockquote> 
   

In [ ]:
%%html
<style>
table {float:left}
</style>

|<p>Field Name|<p>Description|Type|Scale|
|:-----|:-----|:-----|:-----|
|<p>price |<p>The price of the vehicle in US dollars. |integer |ratio |
|<p>price_range |<p>Price category derived from the "price" field, e.g., <1,000, 1,000 - 5, 0000, 5,000 - 10,000, etc. |category |ordinal |
|<p>manufacturer |<p>Manufacturer of the vehicle, e.g., honda, jeep, etc.  There are 44 unique values. |category |nominal |
|<p>model |<p>Model of the vehicle, e.g., f150, malibue, etc.  There are 31,521 unique models. |category |nominal |
|<p>condition_new |<p>Condition of the car with 4 values (salvage, fair_good, excellent_likenew, new).  This field combined some of the values in the original "condition" field. |category |ordinal |
|<p>cylinders |<p>Number of cylinders in the car ranging from 4 to 12. |integer |interval |
|<p>fuel |<p>Type of fuel with the following categories:  'gas', 'diesel', 'other', 'hybrid', 'electric'. |category |nominal |
|<p>odometer |<p>Mileage listed for the vehicle. |integer |ratio |
|<p>title_status |<p>The title status is of the vehicle:  'clean', 'lien', 'missing', 'salvage', 'rebuilt', 'parts only'. |category |nominal |
|<p>transmission_new |<p>Type of transmission:   'other', 'automatic', 'manual'.  This field combined some of the categories from the original 'transmission' field.  The 'other' category includes semi-automatic and dual clutch vehicles. |category |nominal |
|<p>drive |<p>Type of drive: ' rwd', 'fwd', '4wd'. |category |nominal |
|<p>type_new |<p>Type of vehicle with 9 categories after combining some from the original types:  'sedan', 'pickup_truck', 'coupe', 'crossover', 'mini-van','convertible', 'hatchback', 'van', 'offroad'. |category |nominal |
|<p>paint_color_new |<p>Color of the vehicle with 4 categories after combining some from the original colors:   'BlSlvrWhGryMsng', 'BluRdBrwn', 'OrYlwGrnPrpl', 'Custom'. |category |nominal 
|<p>state |<p>State the vehicle is being sold.  All 50 states are represented as well as a category for D.C. |category |nominal |
|<p>car_Age_in_yrs |<p>Derived by subtracting the original year of the vehicle from the date of the Craiglist posting. |integer |ratio |

All the categorical variables will be One Hot Encoded in the [New Features](#New-Features) section below.

## Data Quality
*Verify data quality: Explain any missing values, duplicate data, and outliers. Are those mistakes? How do you deal with these problems? Give justifications for your methods.*

### Duplicate Data (Rudranil)

<blockquote>Duplicate records were in the dataset because the same car could be listed in multiple regions.  The VIN number, which is a unique identifier for vehicles, was used to identify and remove records when possible.  However, there were records that did not contain a VIN and were also listed in multiple regions. After all the data cleaning was finished, a final de-duplication process was run to remove records that contained the same values across all fields.

In [ ]:
#rudranil to add code/notes

### Missing values

<blockquote>There were a significant number of missing values in the original dataset. The first step taken to address this was to attempt to get the missing values from the Department of Transportation's site based on VIN number.  From there, the records that had missing values were dropped or combined into "other" categories.  More details for each field are outlined below.

In [ ]:
#n/a's before getting Dept. of Transportation data
car_df.isna().sum()

<blockquote>Cylinders, year, fuel, drive, transmission, and model were available from the Department of Transportation.  A scraper was built to retrieve the data based on VIN number of records with missing data.  More details about the scraper are in the <a href='#exceptional'>Exceptional section</a>. Once the data was retrieved, it was exported to a separate csv and imported and merged into car_df to fill in missing values by VIN.

In [ ]:
#import VIN dataset
vin_df  = pd.read_csv('data/vin_data.csv',sep='|', encoding='latin-1')

In [ ]:
# Merge the VIN dataset with the original dataset
# With this merge we are able to fill out many missing values in the original dataset 

# Creating a SK Learn Transformer template, will be useful later to generate a 
# data cleaning pipeline 
class MergeTransform(BaseEstimator, TransformerMixin):
    def __init__(self,original_dataset,column_map,columns_to_drop,join_column):
        self.original_dataset = original_dataset
        self.column_map = column_map
        self.columns_to_drop = columns_to_drop
        self.join_column = join_column
    #
    def fit(self, X, y = None):
        return self
    #
    def transform(self, X, y = None):
        #
        X = X.drop_duplicates(['VIN'])
        #
        merged_dataset = pd.merge(self.original_dataset, X, how='left',on=self.join_column)
        #
        for k,v in self.column_map:
            merged_dataset[k].fillna(merged_dataset[v],inplace=True)
        #
        merged_dataset.drop(self.columns_to_drop,axis = 1, inplace = True)
        merged_dataset['cylinders']=merged_dataset['cylinders'].replace({'cylinders':''},regex=True)
        #
        return merged_dataset
    
# The datasets will be joined on VIN
join_col = ['VIN']

# These are the columns that are being cleaned up. The column on the left is the from the cars dataset. These columns
# have null values and need to be replaced from the VIN dataset
column_transform_map = [['cylinders', 'EngineCylinders'],
                        ['year', 'ModelYear'],
                        ['fuel', 'FuelTypePrimary'],
                        ['drive', 'DriveType'],
                        ['transmission', 'TransmissionStyle'],
                        ['model', 'Model']]

# These are additional columns from the VIN dataset an can be dropped after the merge 
columns_to_drop = ['DriveType', 'EngineCylinders', 'EngineManufacturer', 'EngineModel', 'Make', 'Manufacturer', 'Model', 'ModelYear', 'PlantCompanyName', 'TransmissionSpeeds', 'TransmissionStyle', 'FuelTypePrimary']

# Create an instance of the transformer
transformer = MergeTransform(car_df, column_transform_map, columns_to_drop, join_col)

# Run the transformer to generate the clean dataset
car_df = transformer.transform(vin_df)    

<blockquote>After merging the original dataset with the VIN dataset, the missing values have been reduced.

In [ ]:
#n/as after getting Dept. of Transportation data
car_df.isna().sum()

<blockquote>The VIN column was then dropped since it was no longer needed for the analysis.

In [ ]:
#drop extra ID fields, URL fields, region fields, and description
car_df = car_df.drop(columns=['VIN'])

<blockquote>For seven of the columns that still have missing data and no source to obtain it, the rows with missing values were dropped (condition, title_status, drive, fuel, model, manufacturer, type, and year).  The remaining n/a's are combined with other fields in the section below for <a href='#recategorization'>Re-Categorizing Data</a>.

In [ ]:
#drop rows with n/a for important variables that couldn't be obtained from the dept. of transportation
car_df = car_df[car_df['condition'].notna()]
car_df = car_df[car_df['title_status'].notna()]
car_df = car_df[car_df['drive'].notna()]
car_df = car_df[car_df['fuel'].notna()]
car_df = car_df[car_df['model'].notna()]
car_df = car_df[car_df['manufacturer'].notna()]
car_df = car_df[car_df['type'].notna()]
car_df['year'].dropna(inplace=True)

### Outliers

Price, year, and odometer had some major outliers. A lot of the extreme outliers seemed to be mistakes (e.g., 3.5 billion for price, 1900 for year). We implemented a quantile method to screen out the outliers. We set the upper limit for the quantile to 0.99 and the lower limit to 0.01. 

* **Year:** The number of records for year reduced from 457,163 to 441,660 while the IQR (delta between Q1- 25th percentiles and Q3-75th percentiles) stayed the same. The mean of the two is about the same between 2010 and 2011. The left tail of the year distribution pull back significantly from 1900 to 1970 due to this outlier screening. <p>

* **Odometer:** The number of records reduced from 402,910 to 394,622. IQR is quite close between the two (93,123 vs 91,218). The standard deviation is greatly improved from 3,228,623 to 58,661. The max odometer reading changed from 2,043,756,000 (which did not make any sense) to 269,930. <p>

* **Price** The price has several extreme outliers which went as high as $3.6 billion. The count reduced from 458,213 to 419,847. The IQR is extremely wide before outlier screening (210,050 vs 16,002). The standard deviation is improved with outlier screening from 8,194,599 to 11,991.

In [ ]:
#use quantile to remove outliers
q_hi=car_df['price'].quantile(0.99)
q_low=car_df['price'].quantile(0.01)
car_df1 =car_df[(car_df['price']<q_hi)&(car_df['price']>q_low)]

q_hi=car_df1['year'].quantile(0.99)
q_low=car_df1['year'].quantile(0.01)
car_df1 =car_df1[(car_df1['year']<q_hi)&(car_df1['year']>q_low)]

q_hi=car_df1['odometer'].quantile(0.99)
q_low=car_df1['odometer'].quantile(0.01)
car_df1 =car_df1[(car_df1['odometer']<q_hi)&(car_df1['odometer']>q_low)]

<blockquote>Before removing outliers

In [ ]:
car_df[['price','year','odometer']].describe()

In [ ]:
# boxplots before removing the outliers
fig, axs = plt.subplots(ncols=3, figsize=(12,5))
sns.boxplot(y=car_df['price'], ax=axs[0])
sns.boxplot(y=car_df['year'], ax=axs[1])
sns.boxplot(y=car_df['odometer'], ax=axs[2])
plt.show()

<blockquote>After removing outliers

In [ ]:
car_df1[['price','year','odometer']].describe()

In [ ]:
# boxplots after removing the outliers
fig, axs = plt.subplots(ncols=3, figsize=(12,5))
sns.boxplot(y=car_df1['price'], ax=axs[0])
sns.boxplot(y=car_df1['year'], ax=axs[1])
sns.boxplot(y=car_df1['odometer'], ax=axs[2])
plt.show()

### <a id="recategorization">Re-Categorizing Data</a>

<blockquote>The drive type of a vehicle indicates the type of climate and conditions that the vehicle is optimized for.  After retrieving data from the Dept. of Transportation, there were some text mismatches that represented the same types of drives.  After re-categorizing, these are the final categories:<p>
    
    rwd:  Rear-wheel drive 
    4wd:  Four-wheel drive
    fwd:  Front-wheel drive
    others:  As indicated by the seller or when no value was specified.  Glider was categorized here as well.

In [ ]:
# Regroup different drive types
drive_types = [
    (car_df['drive'] == "4WD/4-Wheel Drive/4x4")| (car_df['drive'] == "4wd") | (car_df['drive'] == "4WD/4-Wheel Drive/4x4") |  (car_df['drive'] == "2WD/4WD") | (car_df['drive'] == "6x4") | (car_df['drive'] == "6x2") | (car_df['drive'] == "AWD/All Wheel Drive"),
    (car_df['drive'] == "4x2") | (car_df['drive'] == "2WD/4WD")| (car_df['drive'] == "FWD/Front Wheel Drive")| (car_df['drive'] == "fwd") ,
    (car_df['drive'] == "RWD/ Rear Wheel Drive") | (car_df['drive'] == "rwd"),
    (car_df['drive'] == "Not Applicable") | (car_df['drive'] == "Other") | (car_df['drive'] == "Glider")
     ]
#new drive categories
drive_values = [ '4wd','fwd', 'rwd', 'others']
#new drive column
car_df['drive_new'] = np.select(drive_types, drive_values)
car_df['drive_new'].unique()

<blockquote>One of the goals of this project is to predict categorical responses for price range.  A new category field named "price_range" was derived from the "price" field. These are the final categories:
    
    < 1K
    1K-5K 
    5K-10K 
    10K-15K
    15k-30K 
    30K-50K 
    > 50K
    

In [ ]:
#Create column price_range to classify price of a car into one of the following groups: ('<1K' < '1K-5K' < '5K-10K' < '10K-15K' < '15k-30K' < '30K-50K' < '>50K'etc.)

car_df = car_df1.copy()
car_df['price_range'] = pd.cut(x=car_df['price'], bins=[-1, 1000, 5000,10000,15000,30000,50000, 100000],labels=['<1K', '1K-5K', '5K-10K','10K-15K', '15k-30K','30K-50K', '>50K' ])
car_df['price_range'].unique()

<blockquote>"Condition" of the vehicles is determined by the seller.  Because some of them were very similar, they were re-grouped to a condition_new field with these final categories:
    
    salvage
    good_fair
    excellent_likenew
    new

In [ ]:
#combine conditions to 4 categories with the following groupings ('good', 'fair'), ('excellent','like new'), 'salvage', 'new' 
conditions = [
    (car_df['condition'] == "good") | (car_df['condition'] == "fair"),
    (car_df['condition'] == "excellent") | (car_df['condition'] == "like new"),
    (car_df['condition'] == "salvage"),
    (car_df['condition'] == "new")
    ]

#new condition categories
values = ['good_fair', 'excellent_likenew', 'salvage', 'new']

#new condition column
car_df['condition_new'] = np.select(conditions, values)
car_df['condition_new'].unique()

In [ ]:
#drop the original condition field
car_df = car_df.drop(columns=['condition'])

<blockquote>'Transmission' types had a similar issue to drives in that different text values represented the same thing after the Dept. of Transportation data was retrieved.  This field was regrouped into a 'transmission_new' field with these final categories: automatic, manual, and other. The "other" field consists of transmissions such as Semi-Automatic and Dual-Clutch.

In [ ]:
#Group different transmission categories. 
car_df['transmission'] = car_df['transmission'].fillna('other')
transmission_types = [
    (car_df['transmission'] == "automatic") | (car_df['transmission'] == "Automatic") | (car_df['transmission'] == "Continuously Variable Transmission (CVT)"),
    (car_df['transmission'] == "manual") | (car_df['transmission'] == "Manual/Standard"),
    (car_df['transmission'] == "other")
    ]

#new transmission categories
t_values_types = ['Automatic', 'Manual', 'Other']

#new transmission column
car_df['transmission_new'] = np.select(transmission_types, t_values_types)
car_df['transmission_new'].unique()

In [ ]:
#drop the original transmission field
car_df = car_df.drop(columns=['transmission'])

<blockquote>Vehicle 'type' is a classification given by car makers to represent size and shape.  We felt that several were similar to each other so those were grouped together in a 'type_new' field with these final categories: 'sedan', 'pickup_truck', 'coupe', 'crossover', 'mini-van', 'convertible', 'hatchback', 'van', 'offroad'.

In [ ]:
#Combine types to: 'sedan', ('wagon','SUV') and call it 'crossover', ('pickup', 'truck'), 'coupe', 'van', 'mini-van', 'convertible', 'hatchback', 'offroad'
car_df['type'] = car_df['type'].fillna('')
car_df = car_df[car_df['type'].isin(['sedan', 'SUV', 'pickup', 'coupe', 'van', 'truck','mini-van', 'wagon', 'convertible', 'hatchback','offroad',''])]

types = [
    (car_df['type'] == "wagon") | (car_df['type'] == "SUV"),
    (car_df['type'] == "pickup") | (car_df['type'] == "truck"),
    (car_df['type'] == "sedan"),
    (car_df['type'] == "coupe"),
     (car_df['type'] == "van"),
    (car_df['type'] == "mini-van"),
     (car_df['type'] == "wagon"),
    (car_df['type'] == "convertible"),
     (car_df['type'] == "hatchback"),
    (car_df['type'] == "offroad"),
    (car_df['type'] == "")
    ]
#new condition categories
values_types = ['crossover', 'pickup_truck', 'sedan', 'coupe','van','mini-van','wagon','convertible','hatchback','offroad','']
#new condition column
car_df['type_new'] = np.select(types, values_types)
car_df.head()
car_df['type_new'].unique()

In [ ]:
#drop the original type field
car_df = car_df.drop(columns=['type'])

<blockquote>There were originally 14 paint colors. These were grouped together after looking at boxplots to see which ones were similar to each other.  These are the final categories: 
    
    BlSlvrWhGryMsng: Black, Silver, White, Gray.  N/As are included due to a higher probability of being the more popular colors.
    BluRdBrwn:  Blue, Red, and Brown
    OrYlwGrnPrpl: Orange, Yellow, Green, Purple
    Custom:  Custom paint

In [ ]:
car_df['paint_color'] = car_df['paint_color'].fillna('')
paint_color_grps = [
    (car_df['paint_color'] == "black") | (car_df['paint_color'] == "silver") | (car_df['paint_color'] == "white")|(car_df['paint_color'] == "grey")|(car_df['paint_color'] == '')  ,
    (car_df['paint_color'] == "blue") | (car_df['paint_color'] == "red")| (car_df['paint_color'] == "brown"),
    (car_df['paint_color'] == "orange")| (car_df['paint_color'] == "yellow")| (car_df['paint_color'] == "green")| (car_df['paint_color'] == "purple"),
    (car_df['paint_color'] == "custom")
    ]
#new paint groups
values_paint = ['BlSlvrWhGryMsng', 'BluRdBrwn', 'OrYlwGrnPrpl', 'Custom']
#new condition column
car_df['paint_color_new'] = np.select(paint_color_grps, values_paint)
car_df['paint_color_new'].unique()

In [ ]:
#drop the original color field
car_df = car_df.drop(columns=['paint_color'])

In [ ]:
car_df.info()

### Other cleanup

<blockquote>After the merge with the Dept. of Transportation data, the cylinders were repeated in different formats.  This column was converted to an int with the final values between 2 - 12 cylinders.  1 cyclinder cars were dropped because there were only two records for Honda Pioneers.

In [ ]:
## fill null with Z and remove later
car_df['cylinders'] = car_df['cylinders'].fillna('Z')

In [ ]:
# transform cylinders to integer format
car_df.drop(car_df[car_df['cylinders']=='Z'].index,inplace=True)
car_df.drop(car_df[car_df['cylinders']=='other'].index,inplace=True)
car_df['cylinders'] = car_df['cylinders'].astype('int64')

In [ ]:
# drop 1 cylinder cars
car_df = car_df.drop(car_df[(car_df.cylinders == 1)].index)

In [ ]:
car_df['cylinders'].unique()

<blockquote>Harley-Davidsons were removed since they were motorcycles and this study is focused on cars and trucks.

In [ ]:
# drop "harley-davidson"
car_df = car_df.drop(car_df[(car_df.manufacturer == "harley-davidson")].index)

<blockquote>The "posting_date" was converted to a datetime object to derive a new column for the age of the car (posting_date - year of the car).

In [ ]:
#convert posting_date to datetime object
car_df['posting_date'] = car_df['posting_date'].fillna('')
car_df['posting_date'] = car_df['posting_date'].astype(str)
new_post_Dt= car_df['posting_date'].str.split('T', n = 1, expand = True)
car_df["posting_date"]= new_post_Dt[0]
car_df["posting_date"]=car_df["posting_date"].astype('datetime64[ns]')

In [ ]:
car_df['car_Age_in_yrs'] = car_df['posting_date'].dt.year - car_df['year']
car_df = car_df.drop(columns=['year'])

### Simple Statistics (Rudranil)
Visualize appropriate statistics (e.g., range, mode, mean, median, variance, counts) for a subset of attributes. Describe anything meaningful you found from this or if you found something potentially interesting. Note: You can also use data from other sources for comparison. Explain why the statistics run are meaningful. 

In [ ]:
car_df.describe()

### Visualize Attributes
*Visualize the most interesting attributes (at least 5 attributes, your opinion on what is interesting). Important: Interpret the implications for each visualization. Explain for each attribute why the chosen visualization is appropriate.*

<blockquote>We thought it was interesting to plot some of the data by state. California, Florida, New York are states with the highest number of listings.</blockquote>

In [ ]:
#Reference:  https://github.com/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb

#convert the state column to all upper case
car_df['state'] = car_df['state'].str.upper() 

#import the coordinates of each state
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

#create a new dataframe grouped by state and count of vehicles
grp_df = car_df.groupby(["state"], as_index=False)["price"].count()

# Initialize the map:
m = folium.Map([43, -100], zoom_start=4)

folium.Choropleth(
    geo_data=state_geo,
    data=grp_df,
    columns=["state", "price"],
    key_on="feature.id",
).add_to(m)

m

<blockquote>For Condition of the car, we found it interesting that the excellent_likenew category does not have higher prices than the good_fair category.  This boxplot shows that break out by condition.  The scatterplot on top represent the observations.  From this we can see that the majority of observations in the data set have a condition of excellent_new or good_fair. 

In [ ]:
car_df['condition_new'].value_counts()
#boxplot for condition of car
fig, axs = plt.subplots(ncols=1, figsize=(10,6))
ax = sns.boxplot(x="condition_new", y="price", data=car_df).set(title='Condition & Price', xlabel='Condition', ylabel='Price (USD)')
# Add in points to show each observation
sns.stripplot(x="condition_new", y="price", data=car_df,
              jitter=True, size=2, color="slategrey", linewidth=0)

In [ ]:
#Plot Used cars distribution in by car type
plt.subplots(figsize = (11,7))
car_df['condition_new'].hist(bins=10)
plt.title("Used car distribution by 'Condition' of car", fontsize=18)
plt.xlabel('Car Condition', fontsize=16)
plt.ylabel('Number of Cars', fontsize=16)

In [ ]:
#Plot Used cars distribution in by car type
plt.subplots(figsize = (11,7))
car_df['type_new'].hist(bins=25)
plt.title("Used car distribution by 'Type' of car", fontsize=18)
plt.xlabel('Car Type', fontsize=16)
plt.ylabel('Number of Cars', fontsize=16)

In [ ]:
#Histogram for Numbers of cars in each price range by drive
fig, ax = plt.subplots(figsize = (16,7))
sns.set_theme(style="whitegrid")
plt.title("Car 'Type' distribution by Drive", fontsize=18)
ax = sns.barplot(y="price", x="drive", data=car_df)
ax.set_ylabel('Price ($)', fontsize = 16)
ax.set_xlabel ('Drive', fontsize = 16)
plt.xticks(rotation=45)

### Explore Joint Attributes
*Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.*

<blockquote>Correlations<br>
1) A strong negative correlation between Price and Odometer reading. <br>
2) A strong negative correlation between Price and the age of the car. <br>
3) A strong postive correlation between Odometer reading and the age of the car. <br>
4) A slight positive correlation between the number of cyclinders and the price of the car. <br>
5) A very week correlation between the number of cylinders and the age of the car. <br></blockquote
    

In [ ]:
car_df = car_df.drop(columns=['percent'])
correlations = car_df.corr()
fig, ax = plt.subplots(figsize=(10,8))         # Sample figsize in inches
sns.heatmap(correlations, annot=True, linewidths=.5, ax=ax)

<blockquote> California followed by Florida has the most number of 'excellent_likenew' cars for sale on Craigslist for the time period of this dataset.

In [ ]:
#Plot Used cars distribution by State
fig, ax1 = plt.subplots(figsize = (18,9))
ax1 = sns.histplot(x="state",hue="condition_new", data=car_df)
ax1.set_ylabel('Number of Cars', fontsize = 16)
ax1.set_xlabel ('State', fontsize = 16)
plt.title("Used cars distribution by State", fontsize=18)

<blockquote><b>We observe the following from the plot below:</b><br>

1) For each car type, the cars with the highest odometer reading are in the 'salvage' condition. <br>
2) Off-road vehicles, followed by pickup_trucks and vans have the highest odometer reading for the salvage cars.<br>
3) Convertible cars seem to be inconsistent, with around the same odometer reading, a comparable number of cars are in the 'excellent_likenew' condition or 'salvage' condition.<br>
4) For pick_up trucks, hatchbacks and coupes, the cars in 'excellent_likenew' condition have higher odometer reading compared to these types of cars in 'good/fair' condition. We saw from the plot above that the price for these car types was higher for cars in good/fair condition compared to those in 'excellent/like-new' condition.<p>
Thus condition and car type along with the odometer reading may be good predictors for the price of the car.</blockquote>

In [ ]:
#Distribution of cars by their odometer reading, type and condition
fig, ax = plt.subplots(figsize = (15,7))
sns.set_theme(style="whitegrid")
plt.title("Distribution of the price of the cars by their 'type' and 'odometer' reading", fontsize=18)
ax = sns.barplot(y="odometer",x="type_new", hue="condition_new", data=car_df)
ax.set_ylabel('Odometer', fontsize = 16)
ax.set_xlabel (' Car Type by Condition', fontsize = 16)

### Explore Attributes and Class
Identify and explain interesting relationships between features and the class you are trying to predict (i.e., relationships with variables and the target classification).

#### Features and Price (Regression)


<blockquote>There are two continuous features in the dataset - car age in years and odometer.  Prices are trending down as vehicles age and as the mileage increases.  These features are expected to be useful predictors.
   

In [ ]:
#scatterplots for continuous of car age and odometer by price
fig, axs = plt.subplots(ncols=2, figsize=(18,6))
sns.scatterplot(data=car_df, x="car_Age_in_yrs", y="price", ax=axs[0]).set(title='Age & Price', xlabel='Age (yrs)', ylabel='Price (USD)')
sns.scatterplot(data=car_df, x="odometer", y="price", ax=axs[1]).set(title='Odometer & Price', xlabel='Odometer (miles)', ylabel='Price (USD)')
plt.show()

<blockquote><b>We observe the following from the plot below:</b><p>
1) New cars have the highest prices for cars in each car type. <br>
2) For pickup trucks, hatchbacks and convertible cars, the cars in good_fair condition appear to have higher prices than the cars in excellent_likenew condition for the same car types. Thus, apart from condition and type, there have to be other factors that influence the price of vehicles. <br>
3) Salvage cars have the lowest price for each car type. <br></blockquote>

In [ ]:
#Histogram for Numbers of cars in each price range by car condition
fig, ax = plt.subplots(figsize = (15,7))
sns.set_theme(style="whitegrid")
plt.title("Distribution of the price of the cars by their 'type' and 'condition'", fontsize=18)
ax = sns.barplot(y="price",x="type_new", hue="condition_new", data=car_df)
ax.set_ylabel('Price ($)', fontsize = 16)
ax.set_xlabel (' Car Type by Condition', fontsize = 16)

<blockquote>High-end manufacturers like Aston-Martin, Ferrari, Porshe, and Tesla have higher prices with medians over $30,000. The manufacturers with the lowest median prices are Mercury, Pontiac, and Saturn.  While the median is high for Tesla, there is a large range of values between the 25th PCTL and the 75th PCTL, indicating that there is a lot of variability in prices for this manufacturer.  We expect this to be a useful feature for prediction.

In [ ]:
fig, axs = plt.subplots(ncols=1, figsize=(20,8))
plt.title("Manufacturer and Price", fontsize=18)
ax = sns.boxplot(x="manufacturer", y="price", data=car_df.sort_values("manufacturer"))
ax.set_ylabel('Price ($)', fontsize = 16)
ax.set_xlabel (' Manufacturer', fontsize = 16)
plt.xticks(rotation=90)
plt.show()

<blockquote> There does not seem to be much correlation between state and price. West Virginia and Wyoming an could be an exception to this.

In [ ]:
#Price by State
fig, ax = plt.subplots(figsize = (15,7))
sns.set_theme(style="whitegrid")
plt.title("Price by State", fontsize=18)
ax = sns.boxplot(y="price",x="state", data=car_df)
ax.set_ylabel('Price ($)', fontsize = 16)
ax.set_xlabel ('State', fontsize = 16)

<blockquote> Color seems to have small only a minor correlation with price, while transmission seems to have more correlation.

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20,8))
ax = sns.boxplot(x="paint_color_new", y="price", data=car_df,ax=axs[0])
ax = sns.boxplot(x="transmission_new", y="price", data=car_df,ax=axs[1]) 

#### Categorical Features and Price Range (Classification)

In [ ]:
#Plot Used cars distribution in each price range
plt.subplots(figsize = (11,7))
car_df['price_range'].hist(bins=25)
plt.title('Used cars distribution in each price range', fontsize=18)
plt.xlabel('Price Range', fontsize=16)
plt.ylabel('Number of Cars', fontsize=16)

<blockquote> For classification problems, the prices have been binned into price ranges.  Below are the proportions for each price range for some of the categorical variables. <br> 

The Title Status is a legal document that states the legal owner of the car.  95% of the vehicles in this dataset have a clean title.  More than 50% of these vehicles with clean titles are priced between 10k-50k.  A rebuilt title is for a vehicle that has been damaged or totaled.  More than 50% of the vehicles in this dataset with a rebuilt title are 10k or less.  A lien title is one that has an outstanding loan on the vehicle.  The majority of vehicles in that category are listed between 10k - 50k.  A parts only title indicates that the car cannot be registered again.  The majority of listings for parts only titles are less than 1k.  A missing title can mean that the title was damaged or lost, but could also indicate a stolen car.  Most of the cars in this category are between 1k - 5k.

In [ ]:
# Draw Seaborn Scatter Plot to find relationship between age and fare
fig, axs = plt.subplots(ncols=1,figsize=(15,8))
sns.histplot(car_df, x="title_status", hue="price_range", stat="probability", multiple="fill", shrink=.8)

# Add description for condition

In [ ]:
table=pd.crosstab(car_df['condition_new'],car_df['price_range'])
table.div(table.sum(1).astype(float),axis=0).plot(kind='bar',stacked=True, figsize=(12,6),)

<blockquote>When looking at vehicles by the type of fuel they use, at least 50% or more of them are priced in the 10k-50k range.  92% of the vehicles in this dataset use gas for fuel. The high...

In [ ]:
fig, axs = plt.subplots(ncols=1,figsize=(15,8))
sns.histplot(car_df, x="fuel", hue="price_range", stat="probability", multiple="fill", shrink=.8)

<blockquote>
We created new feature column called “ Car Age” which the tell us the age of the car by substracting posting date from the year column. This new column will help us determine whether there is correlation between the car age and the price of the car which seem definitely it has the correlation. <br>
We also do the one hot encoding on the categorical feature variables i.e. manufacturer(41 distinct),condition new(4 distinct), fuel(5 distinct), title status( 6 distinct), transmission(3 distinct transmission), drive(3 distinct), size( 4 distinct), type(10 distinct type), paint color (4 distinct group color) and cylinder.(8 distinct)

### <a id="New-Features">New Features</a>
*Are there other features that could be added to the data or created from existing features? Which ones?*

* We created a new feature column called “car_Age_in_yrs” which is derived by substracting the age of the car from the posting date. This new column will help determine whether there is correlation between the car age and the price of the car, which we believe to be the case.<p> 
    
* We will also do one hot encoding on all of the the categorical feature variables i.e. manufacturer (41 distinct), condition_new (4 distinct), fuel (5 distinct), title status (6 distinct), transmission(3 distinct transmission), drive(3 distinct), size( 4 distinct), type(10 distinct type), paint_color (4 distinct) and cylinder (8 distinct). <p>
    
* Depending on the results of feature selection, we may reduce the states column to larger regions (west coast, east coast, etc.).

In [ ]:
car_df_encoded = pd.get_dummies(car_df[[ 'price', 'manufacturer', 'model', 'condition_new',
       'fuel', 'odometer', 'title_status', 'transmission_new', 'drive',
       'type_new', 'paint_color_new', 'cylinders']])

<a id='exceptional'></a>

### Exceptional Work

There were columns in the dataset that had missing values. We extracted the information for these columns using the U.S. Department Of Transportation VIN API. (https://vpic.nhtsa.dot.gov/api/). As a valid VIN number is needed to extract information from this API the entire dataset could not be cleanup as many instances were missing VIN number. The following columns were extracted from the VIN API. A scraper was built to call a ReST endpoint and write the details to a file, which was later joined to the original dataset.



| Attribute | Attribute Description | Imputed Column in the Original Dataset |
| --- | --- | --- | 
|VIN | Unique Vehicle Indentifier for a car | Not Mapped. Used for perform join on dataset
| DriveType | Front Wheel Drive, Rear Wheel Drive, All Wheel Drive, etc.| drive
| EngineCylinders | Number of cylinders in the car| cylinders
| EngineManufacturer | Manufacturer Name, for example Ford, GM, etc.| Not Mapped
| EngineModel | The model of the engine | Not Mapped
| Make | Make of the car. For example Toyota, Hyundai, etc. | Not Mapped 
| Manufacturer | Same as engine Manufacturer, but more detailed name | Not Mapped 
| Model | Model of the car. For example Camry, Sonata, etc.| model
| ModelYear | Model when the car was released| year
| PlantCompanyName | Plant where the engine was manufactured| Not Mapped
| TransmissionSpeeds | How many transmissions are present in the car, for Example 4 Speed, 6 Speed etc.|  Not Mapped |
| TransmissionStyle | Kind of transmission. For example, Automatic , Manual, CVT, etc.| transmission |
| FuelTypePrimary | Fuel type used in the car. For example, Gasoline ,Diesel, etc.| fuel |



In [ ]:
def fetch_vin_details(dataset, attribute):
    # List of attributes to filter from the REST response
    vin_attributes = [
        'VIN',
        'DriveType',
        'EngineCylinders',
        'EngineManufacturer',
        'EngineModel',
        'Make',
        'Manufacturer',
        'Model',
        'ModelYear',
        'PlantCompanyName',
        'TransmissionSpeeds',
        'TransmissionStyle',
        'FuelTypePrimary']

    #
    row_list = []
    request_batch_size = 50
    current_fetch_size = 0
    vin_details = ''
    num_requests = 0

    # Create VIN batches so that HTTP requests can be batched to reduce network IO calls for faster processing
    for vin in dataset[attribute]:
        #
        if not pd.isnull(vin):
            #
            vin_details += vin + ';'
            current_fetch_size += 1
            #
            if current_fetch_size == request_batch_size:
                # Introducing a delay to not overload the remote server
                if num_requests == 50:
                    num_requests = 0
                    print(
                        'Going to sleep for sometime to avoid connection refused issue')
                    time.sleep(2)
                # Fetch response from API
                rows = create_http_request(vin_details, vin_attributes)
                vin_details = ''
                current_fetch_size = 0
                num_requests += 1
                row_list.append(rows)

    # Flush out remaining if last fetch size is fewer than the request batch size
    if current_fetch_size < request_batch_size:
        rows = create_http_request(vin_details, vin_attributes)
    row_list.append(rows)

    #
    return row_list


def create_http_request(vin_batch, attributes):
    # construct the batched URL for fetching the VIN information from US DOT
    header = {
        'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'Accept-Encoding': "gzip, deflate, br",
        'Accept-Language': "en-US,en;q=0.5",
        'Connection': "keep-alive",
        'Content-Length': "57",
        'Content-Type': "application/x-www-form-urlencoded",
        'Host': "vpic.nhtsa.dot.gov",
        'Origin': "https://vpic.nhtsa.dot.gov",
        'Referer': "https://vpic.nhtsa.dot.gov/api/",
        'Upgrade-Insecure-Requests': "1",
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0"
    }

    url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/'

    data_dict = {'DATA': vin_batch, 'format': 'JSON'}

    # Parse the raw response in JSON format
    response_raw = requests.post(
        url, headers=header, data=data_dict)

    # Parse if we get a sucessfull response, else return a null list
    if not response_raw.status_code == requests.codes.ok:
        print("Unable to get a reponse at {} for url , {} , {} ".format(
            datetime.now(), url, response_raw.text))
        return []

    #
    # print(response_raw.text)
    response_json = json.loads(response_raw.text)

    with open('vin_data.csv', 'a') as file:
        # Create a dataframe from the JSON response
        rows = []
        for elements in range(0, response_json['Count']):
            values = []
            for key in attributes:
                values.append(response_json['Results'][elements][key])
            rows.append(values)
            file.write("|".join([str(element) for element in values])+"\n")
            file.flush()
    #
    print("Fetched {} records from API at {}".format(len(rows), datetime.now()))
    file.close()
    #
    return rows


def unpack_list_to_df(row_list):
    #
    df_list = []

    for indx in range(0, len(row_list)):
        for iter in range(0, len(row_list[indx])):
            df_list.append(row_list[indx][iter])

    #
    return pd.DataFrame(df_list, columns=[
        'VIN',
        'DriveType',
        'EngineCylinders',
        'EngineManufacturer',
        'EngineModel',
        'Make',
        'Manufacturer',
        'Model',
        'ModelYear',
        'PlantCompanyName',
        'TransmissionSpeeds',
        'TransmissionStyle',
        'FuelTypePrimary'])